Adaptation of scraper tool: https://medium.com/c%C3%B3digo-ecuador/how-to-scrape-yahoo-price-history-data-with-python-52751eee9b

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import lxml
from lxml import html

# Set-up

*Datetime* objects can't be used in a URL, so **format_date** first takes a datetime object as an argument and then breaks it down to a *timetuple*. From there, we'll convert the tuple into a string of time in seconds passed since epoch.

Using **subdomain**, I can search for any organization's stock price history as long as I have their stock ticker symbol. The start and end time parameters are provided by the prior.

**header_function** provides the appropriate headers when accessing Yahoo's API. These pointers tell Yahoo exactly what page we are trying to access and prevents your machine from being blocked for requests.

In [2]:
def format_date(date_datetime):
    date_timetuple = date_datetime.timetuple()
    date_mktime = str(int(time.mktime(date_timetuple)))
    return date_mktime

def subdomain(symbol, start, end, filter='history'):
    subdomain = f'/quote/{symbol}/history?period1={start}&period2={end}&interval=1d&filter={filter}&frequency=1d'
    return subdomain
    
def header_function(subdomain):
    hdrs =  {"authority": "finance.yahoo.com",
               "method": "GET",
               "path": subdomain,
               "scheme": "https",
               "accept": "text/html",
               "accept-encoding": "gzip, deflate, br",
               "accept-language": "en-US,en;q=0.9",
               "cache-control": "no-cache",
               "cookie": "Cookie:identifier",
               "dnt": "1",
               "pragma": "no-cache",
               "sec-fetch-mode": "navigate",
               "sec-fetch-site": "same-origin",
               "sec-fetch-user": "?1",
               "upgrade-insecure-requests": "1",
               "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64)"}
    return hdrs

# Scraping

**scrape_page** implements the lxml module for scraping. Provided with the URL and headers, we'll first set up a request. Next, we'll convert the contents of the response to HTML format and navigate through the tags using XPath. For Yahoo Finance, our data of interest is found within a *table* tag as the first subelement. In **line 8**, we convert the contents into a string representation of an HTML table and finally read that into a *DataFrame* using *pandas*.

In [3]:
def scrape_page(url, header):
    page = requests.get(url, headers=header)
    
    element_html = html.fromstring(page.content)
    
    table = element_html.xpath('//table')
    
    table_tree = lxml.etree.tostring(table[0], method='xml')
    
    df = pd.read_html(table_tree)
    
    return df

# Requests Limit Bypass

Yahoo restricts requests to 100 days of historical stock data, so I've designed a loop that will allow us to retrieve the complete set of Tesla stock prices since their IPO on June 29th, 2010.

In [4]:
# Stock Ticker
symbol = 'TSLA'

# Using the datetime module and today() method, I can retrieve the most up-to-date stock prices at any given time by
# running this script.
dt_start = datetime.today() - timedelta(days = 100) # We "start" scraping from 100 days ago from today.
dt_end = datetime.today() # We "stop" scraping once we reach today's data.
 
# price_history is a list of DataFrames, each containing iterations of 100-days of data. 
price_history = []

for _ in range(45):
    start = format_date(dt_start)
    end = format_date(dt_end)
    
    sub = subdomain(symbol, start, end)
    header = header_function(sub)

    base_url = 'https://finance.yahoo.com/'
    url = base_url + sub
    price_history += scrape_page(url, header)
    dt_end = dt_start
    dt_start -= timedelta(days = 100) # This line resets our start and end dates shifting by 100-day windows.`

In [5]:
price_history[0].head() # The first element of our list is the last 100 days from today's date.

Date             Open             High              Low  \
0  Sep 03, 2020           407.23           431.80           402.00   
1  Sep 02, 2020           478.99           479.04           405.12   
2  Sep 01, 2020           502.14           502.49           470.51   
3  Aug 31, 2020           444.61           500.14           440.11   
4  Aug 31, 2020  5:1 Stock Split  5:1 Stock Split  5:1 Stock Split   

            Close*      Adj Close**           Volume  
0           407.00           407.00         85937700  
1           447.37           447.37         96176100  
2           475.05           475.05         90119400  
3           498.32           498.32        118374400  
4  5:1 Stock Split  5:1 Stock Split  5:1 Stock Split

# Final DataFrame

In **line 1** below, I've created a new list of *DataFrames* that have the "Date" column set as the index. Finally, I concatenate the list of *DataFrames* to output the final version to be used in my analysis.

In [6]:
df = pd.concat([df.set_index('Date') for df in price_history])
df.drop(df.tail(5).index, inplace = True)
df

Open             High              Low  \
Date                                                              
Sep 03, 2020           407.23           431.80           402.00   
Sep 02, 2020           478.99           479.04           405.12   
Sep 01, 2020           502.14           502.49           470.51   
Aug 31, 2020           444.61           500.14           440.11   
Aug 31, 2020  5:1 Stock Split  5:1 Stock Split  5:1 Stock Split   
...                       ...              ...              ...   
Jul 06, 2010             4.00             4.00             3.17   
Jul 02, 2010             4.60             4.62             3.74   
Jul 01, 2010             5.00             5.18             4.05   
Jun 30, 2010             5.16             6.08             4.66   
Jun 29, 2010             3.80             5.00             3.51   

                       Close*      Adj Close**           Volume  
Date                                                             
Sep 03, 2020           407.00           407.00         85937700  
Sep 02, 2020           447.37           447.37         96176100  
Sep 01, 2020           475.05           475.05         90119400  
Aug 31, 2020           498.32           498.32        118374400  
Aug 31, 2020  5:1 Stock Split  5:1 Stock Split  5:1 Stock Split  
...                       ...              ...              ...  
Jul 06, 2010             3.22             3.22         34334500  
Jul 02, 2010             3.84             3.84         25699000  
Jul 01, 2010             4.39             4.39         41094000  
Jun 30, 2010             4.77             4.77         85935500  
Jun 29, 2010             4.78             4.78         93831500  

[2566 rows x 6 columns]

In [7]:
df.to_csv('TSLA.csv', index = True) # Save to csv for record-keeping and importing.